In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from src.DataSource import DataSource

In [2]:
filename="./data/titanic_train.csv"
ds_titanic = DataSource(filename)

ds_titanic.data_load_split(target=['Survived'], 
                   ignore=["Name", "Cabin", "Ticket"])
ds_titanic.define_problem()
ds_titanic.train_val_split(ratio=0.2, random_state=42)
ds_titanic.data_preprocess(ds_titanic.X_train, ds_titanic.y_train, train_set=True)
ds_titanic.data_preprocess(ds_titanic.X_val, ds_titanic.y_val, train_set=False)

### Whether made from Functional APi vs Sequential, Is it same or not?
- result: `same`

In [3]:
def get_mlp_with_api():
    tf.random.set_seed(42)
    inputs = keras.Input(shape=(ds_titanic.trans_X_train.shape[1]))
    x = keras.layers.Dense(16, activation='relu')(inputs)
    x = keras.layers.Dense(16, activation='relu')(x)
    outputs = keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [14]:
model1 = get_mlp_with_api()
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 662)]             0         
_________________________________________________________________
dense_15 (Dense)             (None, 16)                10608     
_________________________________________________________________
dense_16 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 17        
Total params: 10,897
Trainable params: 10,897
Non-trainable params: 0
_________________________________________________________________


In [11]:
def get_mlp_with_sequential():
    tf.random.set_seed(42)
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(ds_titanic.trans_X_train.shape[1],)))
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    return model

In [15]:
model2 = get_mlp_with_sequential()
model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 16)                10608     
_________________________________________________________________
dense_19 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 17        
Total params: 10,897
Trainable params: 10,897
Non-trainable params: 0
_________________________________________________________________


In [16]:
model1.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])

model2.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [17]:
callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model1.fit(x=ds_titanic.trans_X_train, y=ds_titanic.trans_y_train,
          batch_size=64, epochs=10,
          verbose=0, callbacks=[callbacks],
          validation_data=(ds_titanic.trans_X_val, ds_titanic.trans_y_val), shuffle=True)

model1.evaluate(ds_titanic.trans_X_val, ds_titanic.trans_y_val)


model2.fit(x=ds_titanic.trans_X_train, y=ds_titanic.trans_y_train,
          batch_size=64, epochs=10,
          verbose=0, callbacks=[callbacks],
          validation_data=(ds_titanic.trans_X_val, ds_titanic.trans_y_val), shuffle=True)

model2.evaluate(ds_titanic.trans_X_val, ds_titanic.trans_y_val)

210/210 [==============================] - 0s 77us/sample - loss: 0.6886 - accuracy: 0.5857


[0.6885807741255987, 0.5857142857142857]

### If making the same model repeatedly, are the results same?
- result: `same`

In [40]:
def get_mlp_with_sequential_times(times=3):
    models = []
    for _ in range(times):
        tf.random.set_seed(42)
        model = keras.Sequential()
        model.add(keras.layers.Input(shape=(ds_titanic.trans_X_train.shape[1],)))
        model.add(keras.layers.Dense(16, activation='relu'))
        model.add(keras.layers.Dense(16, activation='relu'))
        model.add(keras.layers.Dense(1, activation='sigmoid'))
        models.append(model)
    return models

In [41]:
models = get_mlp_with_sequential_times(3)

In [42]:
for model in models:
    model.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [43]:
for model in models:
    callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(x=ds_titanic.trans_X_train, y=ds_titanic.trans_y_train,
          batch_size=64, epochs=10,
          verbose=0, callbacks=[callbacks],
          validation_data=(ds_titanic.trans_X_val, ds_titanic.trans_y_val), shuffle=True)

In [44]:
for model in models:
    model.evaluate(ds_titanic.trans_X_val, ds_titanic.trans_y_val)

210/210 [==============================] - 0s 99us/sample - loss: 0.6886 - accuracy: 0.5857


### If making the 2-layered-mlp with iteration or manually, are the results same?
- result: `same`
- Need to develop a function to create a model
- And using the function repeatedly to develop several models with many different structures.

In [55]:
def get_mlp_with_sequential_auto(hidden=2):
    tf.random.set_seed(42)
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(ds_titanic.trans_X_train.shape[1],)))
    for _ in range(hidden):
#         tf.random.set_seed(42)
        model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    return model

In [56]:
model1 = get_mlp_with_sequential_auto()
model1.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 16)                10608     
_________________________________________________________________
dense_70 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 17        
Total params: 10,897
Trainable params: 10,897
Non-trainable params: 0
_________________________________________________________________


In [57]:
model2 = get_mlp_with_sequential()
model2.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 16)                10608     
_________________________________________________________________
dense_73 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_74 (Dense)             (None, 1)                 17        
Total params: 10,897
Trainable params: 10,897
Non-trainable params: 0
_________________________________________________________________


In [58]:
for model in [model1, model2]:
    model.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])
for model in [model1, model2]:
    callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(x=ds_titanic.trans_X_train, y=ds_titanic.trans_y_train,
          batch_size=64, epochs=10,
          verbose=0, callbacks=[callbacks],
          validation_data=(ds_titanic.trans_X_val, ds_titanic.trans_y_val), shuffle=True)
for model in [model1, model2]:
    model.evaluate(ds_titanic.trans_X_val, ds_titanic.trans_y_val)

210/210 [==============================] - 0s 81us/sample - loss: 0.6886 - accuracy: 0.5857


### MLP model that user can build it's structure

In [67]:
def get_mlp(hidden=1, unit=16):
    tf.random.set_seed(42)
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(ds_titanic.trans_X_train.shape[1],)))
    for _ in range(hidden):
        model.add(keras.layers.Dense(unit, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    return model

In [64]:
def get_structures(max_hidden=1, units=[16], use_all=False):
    if use_all:
        max_hidden = 3
        units = [16, 32, 64, 128, 256]
    else:
        max_hidden = max_hidden
        units = units
        
    grid = [np.arange(max_hidden)+1, units]
    return grid

In [70]:
grid = get_structures(max_hidden=2, units=[16])
grid

[array([1, 2]), [16]]

In [72]:
import itertools

def build_mlps(grid):
    models = []
    for param_tuple in itertools.product(*grid):
        structure_param = {'hidden_layers': param_tuple[0],
                           'units': param_tuple[1]}
        
        model = get_mlp(hidden=structure_param['hidden_layers'], unit=structure_param['units'])
        models.append(model)
    return models

In [73]:
models = build_mlps(grid)

In [79]:
for model in models:
    model.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])
for model in models:
    callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(x=ds_titanic.trans_X_train, y=ds_titanic.trans_y_train,
          batch_size=64, epochs=10,
          verbose=0, callbacks=[callbacks],
          validation_data=(ds_titanic.trans_X_val, ds_titanic.trans_y_val), shuffle=True)
for model in models:
    model.evaluate(ds_titanic.trans_X_val, ds_titanic.trans_y_val)

210/210 [==============================] - 0s 113us/sample - loss: 0.6886 - accuracy: 0.5857


In [81]:
# single models
model_1 = get_mlp(hidden=1, unit=16)
model_2 = get_mlp(hidden=2, unit=16)

In [83]:
models_1 = [model_1, model_2]
for model in models_1:
    model.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])
for model in models_1:
    callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(x=ds_titanic.trans_X_train, y=ds_titanic.trans_y_train,
          batch_size=64, epochs=10,
          verbose=0, callbacks=[callbacks],
          validation_data=(ds_titanic.trans_X_val, ds_titanic.trans_y_val), shuffle=True)
for model in models_1:
    model.evaluate(ds_titanic.trans_X_val, ds_titanic.trans_y_val)

210/210 [==============================] - 0s 145us/sample - loss: 0.6886 - accuracy: 0.5857
